- DB
    - id : 00000(int64)
    - 서비스ID : 'OA-00000'(str)
    - 서비스명 : (str)
    - 서비스 설명 : (str)


- Faiss
    - id : 00000(int64)
    - index : vector


- Faiss 인덱스 업데이트 및 저장
- Faiss 인덱스 불러오기
- Faiss 유사도 검색
    - Top 5
        - input : faiss id
    - Top 1


In [1]:
import pandas as pd
import numpy as np
import faiss

vector_sample = np.random.rand(20, 32).astype('float32')

# print(vector_sample[0][:10])

# index = faiss.IndexFlatL2(vector_sample.shape[1])
# print(index.ntotal)

# index.add(vector_sample)
# print(index.ntotal)

ids = np.array([a for a in range(1, 21)]).astype('int64')

index = faiss.IndexFlatL2(vector_sample.shape[1])
index = faiss.IndexIDMap2(index)
index.add_with_ids(vector_sample, ids)
d, i = index.search(vector_sample[0].reshape(1, -1), 3)
print(d)
print(i)

In [1]:
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np
import torch
import pandas as pd

/Users/ktg/miniforge3/envs/fastapi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# load data
df = pd.read_csv("../Crawling/서울시 공공데이터 최종.csv")
# df.head().T

# id = df['서비스ID'].to_list()[:100]
# id_int = np.array([int(i.split('-')[1]) for i in id]).astype('int64')

id_int = np.array( range(0, 100), dtype='int64' )
title = df['서비스명'].to_list()[:100]
df
# service_dscrp = df['서비스설명'].to_list()[:100]

,서비스ID,서비스명,대분류,중분류,소분류,제공기관,제공부서명,시스템명,담당자명,담당자연락처,갱신주기,최종갱신일자,제공사이트,제공형식,서비스설명,urls
0,OA-12615,서울시 강동구 명일2동 주민센터 새소식 정보,공공데이터,자치구 및 자치구산하,일반행정,강동구,강동구 명일2동,강동구 홈페이지,강동구 대표전화,02-1577-1188,일간,NaN,강동구 홈페이지(https://www.gangdong.go.kr/web/dongre...,"Sheet,Api","명일2동 주민센터에서 주민에게 알리는 소식 정보로서 게시물ID, 게시물순번, 게시물...",https://data.seoul.go.kr/dataList/OA-12615/S/1...
1,OA-12621,서울시 강동구 암사2동 주민센터 새소식 정보,공공데이터,자치구 및 자치구산하,일반행정,강동구,강동구 암사2동,강동구 홈페이지,강동구 대표전화,02-1577-1188,일간,NaN,강동구 홈페이지(https://www.gangdong.go.kr/web/dongre...,"Sheet,Api","암사2동 주민센터에서 주민에게 알리는 소식 정보로서 게시물ID, 게시물순번, 게시물...",https://data.seoul.go.kr/dataList/OA-12621/S/1...
2,OA-12611,서울시 강동구 길동 주민센터 새소식 정보,공공데이터,자치구 및 자치구산하,일반행정,강동구,강동구 길동,강동구 홈페이지,강동구 대표전화,02-1577-1188,일간,NaN,강동구 홈페이지(https://www.gangdong.go.kr/web/dongre...,"Sheet,Api","길동 주민센터에서 주민에게 알리는 소식 정보로서 게시물ID, 게시물순번, 게시물제목...",https://data.seoul.go.kr/dataList/OA-12611/S/1...
3,OA-18805,서울시 성동구 통신판매업 인허가 정보,공공데이터,서울시(본청),산업/경제,성동구,스마트도시정책관 빅데이터담당관,지방행정 인허가 데이터개방,성동구 대표전화,02-2286-5114,수시,2023-04-14,지방행정 인허가 데이터개방(http://localdata.kr),"Sheet,Api","성동구의 TV홈쇼핑, 인터넷, 카탈로그, 신문잡지, 기타의 방법으로 가구, 가전, ...",https://data.seoul.go.kr/dataList/OA-18805/S/1...
4,OA-19284,서울시 성북구 건물위생관리업 인허가 정보,공공데이터,서울시(본청),환경,성북구,스마트도시정책관 빅데이터담당관,지방행정 인허가 데이터개방,성북구 대표전화,02-2241-3114,수시,2023-04-14,지방행정 인허가 데이터개방(http://localdata.kr),"Sheet,Api",성북구의 공중이 이용하는 건축물·시설물등의 청결유지와 실내공기정화를 위한 청소 등을...,https://data.seoul.go.kr/dataList/OA-19284/S/1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7393,OA-350,서울시 연평균최고기온 2005년 위치정보 (좌표계: ITRF2000),공공데이터,서울시(본청),환경,서울특별시,기후환경본부 대기정책과,서울시 기후.에너지 지도(2010),이경옥,02-2133-3597,주기없음,2012-01-13,서울시 기후.에너지 지도(2010),"MAP,File,Api,Sheet,Chart","<b>※ 본 데이터는 일회성 데이터로, 최신 데이터는 보유하고 있지 않으니 이용에 ...",https://data.seoul.go.kr/dataList/OA-350/S/1/d...
7394,OA-357,서울시 연평균최저기온 2008년 위치정보 (좌표계: ITRF2000),공공데이터,서울시(본청),환경,서울특별시,기후환경본부 대기정책과,서울시 기후.에너지 지도(2010),이경옥,02-2133-3597,주기없음,2012-01-13,서울시 기후.에너지 지도(2010),"MAP,File,Sheet,Api,Chart","<b>※ 본 데이터는 일회성 데이터로, 최신 데이터는 보유하고 있지 않으니 이용에 ...",https://data.seoul.go.kr/dataList/OA-357/S/1/d...
7395,OA-342,서울시 자치구별 TOE 환산량 2008년 위치정보 (좌표계: ITRF2000),공공데이터,서울시(본청),산업/경제,서울특별시,기후환경본부 대기정책과,서울시 기후.에너지 지도(2010),이경옥,02-2133-3597,주기없음,2012-01-13,서울시 기후.에너지 지도(2010),"MAP,File,Sheet,Api,Chart","<b>※ 본 데이터는 일회성 데이터로, 최신 데이터는 보유하고 있지 않으니 이용에 ...",https://data.seoul.go.kr/dataList/OA-342/S/1/d...
7396,OA-330,서울시 행정동별 전력 사용량 2008년 위치정보 (좌표계: ITRF2000),공공데이터,서울시(본청),산업/경제,서울특별시,기후환경본부 대기정책과,서울시 기후.에너지 지도(2010),이경옥,02-2133-3597,주기없음,2012-01-13,서울시 기후.에너지 지도(2010),"MAP,File,Sheet,Api","<b>※ 본 데이터는 일회성 데이터로, 최신 데이터는 보유하고 있지 않으니 이용에 ...",https://data.seoul.go.kr/dataList/OA-330/S/1/d...


In [3]:
# init
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModel.from_pretrained("klue/bert-base")
index = None
text_db = []

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# get embeddings
def get_embeddings(texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
        embeddings = model_output.last_hidden_state[:, 0, :].numpy()
    return embeddings.astype('float32')

embeddings = get_embeddings(title)

In [5]:
print(embeddings.shape)
print(id_int.shape)

(100, 768)
(100,)


In [6]:
# build
index = faiss.IndexFlatL2(embeddings.shape[1])
index = faiss.IndexIDMap2(index)

index.add_with_ids(embeddings, id_int)

In [7]:
# search
query = "검색어를 입력하세요"
query_embedding = get_embeddings([query])
print(query_embedding.shape)
print(query_embedding.reshape(1, -1).shape)

(1, 768)
(1, 768)


In [8]:
D, I = index.search(query_embedding, 3)
print(D, I)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np
import torch
import pandas as pd

# load data
df = pd.read_csv("../Crawling/서울시 공공데이터 최종.csv")
# df.head().T

id = df['서비스ID'].to_list()[:100]
id_int = [int(i.split('-')[1]) for i in id]
title = df['서비스명'].to_list()[:100]
# service_dscrp = df['서비스설명'].to_list()[:100]

# init
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModel.from_pretrained("klue/bert-base")
index = None
text_db = []

# embeddings
def get_embeddings(texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
        embeddings = model_output.last_hidden_state[:, 0, :].numpy()
    return embeddings

embeddings = get_embeddings(title)

# build
index = faiss.IndexFlatL2(embeddings.shape[1])
index = faiss.IndexIDMap2(index)

index.add_with_ids(embeddings, id_int)

distances, indices = index.search(embeddings[0].reshape(1, -1), 3)
print(distances, indices)

In [6]:
def search_text(query, k=5):
    if index is None:
        raise ValueError("Index is not built yet. Call build_index() first.")
    
    query_embedding = get_embeddings([query])[0]
    distances, indices = index.search(np.array([query_embedding]), k)
    
    print('distances')
    print(distances)
    print('indices')
    print(indices)
    
    results = [(text_db[idx], score) for idx, score in zip(indices[0], distances[0])]
    return results

In [11]:
query = '서울특별시'
k=3
query_embedding = get_embeddings([query])[0]

# query_embedding.reshape(1, -1)
distances, indices = index.search(query_embedding.reshape(1, -1), k)


# index.search()

In [19]:
vector_sample = np.random.rand(20, 32).astype('float32')
vector_sample.shape[1]

32

In [1]:
from faiss_index import FaissIndex

/Users/ktg/miniforge3/envs/fastapi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
index = FaissIndex(index_file_path='index.faiss')

# 인덱스를 로드
index.load_index()

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


True

In [5]:
# 검색 예시
query = '지하철'
result = index.search_query(query=query, k=5)
print(result)